In [1]:
# Imports
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
# Train data needs Data_Raw, train_data, X_train, y_train
# Test data needs Data_Raw, test_data, X_test, y_test
def data_input(folder1, folder2, file_x, file_y):
    X = pd.read_csv(f"../Accident_Data_Preprocessing/{folder1}/{folder2}/{file_x}.csv")
    y = pd.read_csv(f"../Accident_Data_Preprocessing/{folder1}/{folder2}/{file_y}.csv")
    df = pd.concat([X, y], axis=1)
    return df

In [3]:
# TEST INPUT
#df = data_input("Data_Raw", "test_data", "X_test", "y_test")
#df.head()

In [4]:
# TRAIN INPUT
df = data_input("Data_Raw", "train_data", "X_train", "y_train")
df.head()

,ID,Source,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity
0,A-1582799,Source2,2020-01-09 21:03:06,2020-01-09 21:30:52,38.909382,-76.725899,NaN,NaN,0.000,Accident on Pointer Ridge Dr at Philmont Ln.,...,False,False,False,False,False,Night,Night,Night,Night,2
1,A-480671,Source2,2017-06-08 08:01:59,2017-06-08 08:46:37,39.287880,-76.533447,NaN,NaN,0.000,Right hand shoulder closed due to accident on ...,...,False,False,False,False,False,Day,Day,Day,Day,3
2,A-259111,Source2,2016-07-14 17:13:19,2016-07-14 18:43:19,39.109459,-77.160103,NaN,NaN,0.000,Accident on MD-355 Frederick Rd at MD-355 Fred...,...,False,False,False,False,False,Day,Day,Day,Day,2
3,A-3629107,Source1,2017-06-13 07:14:33,2017-06-13 13:14:33,39.391690,-77.322670,39.39387,-77.3265,0.254,At MD-144/Exit 59 - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,2
4,A-3051370,Source2,2018-01-08 07:48:00,2018-01-08 08:16:31,39.387390,-77.143936,NaN,NaN,0.000,Accident on MD-808 Main St Northbound at Ellis...,...,False,False,False,False,False,Day,Day,Day,Day,2


In [5]:
# View data, remove duplicate rows
df.head()
df = df.drop_duplicates()

In [6]:
# Check all unique values for all columns
for col in df.columns:
    print(df[col].value_counts(sort=True))

ID
A-7696245    1
A-1582799    1
A-480671     1
A-259111     1
A-3629107    1
            ..
A-4119726    1
A-259464     1
A-7137842    1
A-7093950    1
A-5039960    1
Name: count, Length: 112333, dtype: int64
Source
Source1    78740
Source2    33071
Source3      522
Name: count, dtype: int64
Start_Time
2021-05-28 07:31:48    22
2022-05-18 07:05:30    20
2021-11-06 16:59:00    17
2021-10-18 10:31:30    15
2022-01-21 12:38:30    14
                       ..
2021-08-04 17:05:47     1
2022-07-07 07:51:48     1
2022-01-01 13:16:12     1
2018-05-10 08:04:46     1
2017-07-05 14:18:17     1
Name: count, Length: 96385, dtype: int64
End_Time
2021-02-22 12:48:14    10
2021-01-29 06:22:11     8
2021-10-27 09:14:30     8
2022-07-15 15:23:55     8
2020-04-30 16:37:30     8
                       ..
2018-03-12 14:08:19     1
2021-08-04 19:08:26     1
2022-07-07 08:21:23     1
2022-01-01 15:46:13     1
2016-10-21 18:11:44     1
Name: count, Length: 102903, dtype: int64
Start_Lat
38.971164    164
39.4

In [7]:
# Worth keeping weather time stamp? Examine value types
print(df["Weather_Timestamp"].value_counts(sort=True))

Weather_Timestamp
2022-03-13 01:55:00    55
2022-05-18 06:52:00    25
2021-05-28 07:58:00    25
2021-11-06 16:55:00    21
2021-02-22 10:54:00    21
                       ..
2023-01-01 16:55:00     1
2021-02-21 15:55:00     1
2022-08-13 17:52:00     1
2017-08-02 20:53:00     1
2021-11-26 07:47:00     1
Name: count, Length: 61197, dtype: int64


In [8]:
# Check all values of all remaining cols
for col in df.columns:
    print(df[col].value_counts(sort=True))

ID
A-7696245    1
A-1582799    1
A-480671     1
A-259111     1
A-3629107    1
            ..
A-4119726    1
A-259464     1
A-7137842    1
A-7093950    1
A-5039960    1
Name: count, Length: 112333, dtype: int64
Source
Source1    78740
Source2    33071
Source3      522
Name: count, dtype: int64
Start_Time
2021-05-28 07:31:48    22
2022-05-18 07:05:30    20
2021-11-06 16:59:00    17
2021-10-18 10:31:30    15
2022-01-21 12:38:30    14
                       ..
2021-08-04 17:05:47     1
2022-07-07 07:51:48     1
2022-01-01 13:16:12     1
2018-05-10 08:04:46     1
2017-07-05 14:18:17     1
Name: count, Length: 96385, dtype: int64
End_Time
2021-02-22 12:48:14    10
2021-01-29 06:22:11     8
2021-10-27 09:14:30     8
2022-07-15 15:23:55     8
2020-04-30 16:37:30     8
                       ..
2018-03-12 14:08:19     1
2021-08-04 19:08:26     1
2022-07-07 08:21:23     1
2022-01-01 15:46:13     1
2016-10-21 18:11:44     1
Name: count, Length: 102903, dtype: int64
Start_Lat
38.971164    164
39.4

In [ ]:
# Date/time conversion of start and end time
df['Start_Time'] = pd.to_datetime(df['Start_Time'],format='mixed')
df['End_Time'] = pd.to_datetime(df['End_Time'], format = 'mixed')
df

,ID,Source,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity
0,A-1582799,Source2,2020-01-09 21:03:06,2020-01-09 21:30:52,38.909382,-76.725899,NaN,NaN,0.000,Accident on Pointer Ridge Dr at Philmont Ln.,...,False,False,False,False,False,Night,Night,Night,Night,2
1,A-480671,Source2,2017-06-08 08:01:59,2017-06-08 08:46:37,39.287880,-76.533447,NaN,NaN,0.000,Right hand shoulder closed due to accident on ...,...,False,False,False,False,False,Day,Day,Day,Day,3
2,A-259111,Source2,2016-07-14 17:13:19,2016-07-14 18:43:19,39.109459,-77.160103,NaN,NaN,0.000,Accident on MD-355 Frederick Rd at MD-355 Fred...,...,False,False,False,False,False,Day,Day,Day,Day,2
3,A-3629107,Source1,2017-06-13 07:14:33,2017-06-13 13:14:33,39.391690,-77.322670,39.393870,-77.326500,0.254,At MD-144/Exit 59 - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,2
4,A-3051370,Source2,2018-01-08 07:48:00,2018-01-08 08:16:31,39.387390,-77.143936,NaN,NaN,0.000,Accident on MD-808 Main St Northbound at Ellis...,...,False,False,False,False,False,Day,Day,Day,Day,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112328,A-7557011,Source1,2018-07-05 15:05:28,2018-07-05 21:05:28,39.401610,-76.563460,39.401740,-76.567870,0.236,At MD-542/Lock Raven Blvd/Exit 29 - Accident. ...,...,False,False,False,False,False,Day,Day,Day,Day,3
112329,A-3629913,Source1,2017-06-15 14:33:24,2017-06-15 20:33:24,38.819500,-76.917669,38.820156,-76.922780,0.279,At MD-5/Branch Ave/Exit 7 - Accident.,...,False,False,False,False,False,Day,Day,Day,Day,3
112330,A-381693,Source2,2017-03-14 04:31:38,2017-03-14 05:18:32,38.960472,-76.919800,NaN,NaN,0.010,Accident on MD-201 Kenilworth Ave Southbound a...,...,False,False,False,True,False,Night,Night,Night,Night,2
112331,A-7463123,Source1,2019-01-15 08:49:03,2019-01-15 09:18:29,39.026619,-76.615523,39.021964,-76.615013,0.323,At US-50/US-301 - Accident. Lane blocked.,...,False,False,False,False,False,Day,Day,Day,Day,2


In [10]:
# Check unique values for all cols again
for col in df.columns:
    print(df[col].value_counts(sort=True))

ID
A-7696245    1
A-1582799    1
A-480671     1
A-259111     1
A-3629107    1
            ..
A-4119726    1
A-259464     1
A-7137842    1
A-7093950    1
A-5039960    1
Name: count, Length: 112333, dtype: int64
Source
Source1    78740
Source2    33071
Source3      522
Name: count, dtype: int64
Start_Time
2022-05-18 07:05:30    25
2021-05-28 07:31:48    25
2021-11-06 16:59:00    20
2022-04-23 18:32:30    19
2022-01-21 12:38:30    19
                       ..
2019-01-16 06:37:46     1
2022-07-17 02:30:30     1
2017-09-28 17:25:27     1
2020-10-24 11:31:30     1
2019-07-31 19:51:59     1
Name: count, Length: 92305, dtype: int64
End_Time
2022-07-15 15:23:55    11
2021-02-22 12:48:14    10
2021-01-29 06:22:11     8
2022-01-03 12:22:02     8
2020-04-30 16:37:30     8
                       ..
2017-09-28 17:55:06     1
2020-10-24 13:34:01     1
2018-03-12 14:08:19     1
2021-08-04 19:08:26     1
2019-12-09 12:13:12     1
Name: count, Length: 100590, dtype: int64
Start_Lat
38.971164    164
39.4

In [ ]:
# Create month column out of date time
df['Month'] = df['Start_Time'].dt.month
df

,ID,Source,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,Month
0,A-1582799,Source2,2020-01-09 21:03:06,2020-01-09 21:30:52,38.909382,-76.725899,NaN,NaN,0.000,Accident on Pointer Ridge Dr at Philmont Ln.,...,False,False,False,False,Night,Night,Night,Night,2,1
1,A-480671,Source2,2017-06-08 08:01:59,2017-06-08 08:46:37,39.287880,-76.533447,NaN,NaN,0.000,Right hand shoulder closed due to accident on ...,...,False,False,False,False,Day,Day,Day,Day,3,6
2,A-259111,Source2,2016-07-14 17:13:19,2016-07-14 18:43:19,39.109459,-77.160103,NaN,NaN,0.000,Accident on MD-355 Frederick Rd at MD-355 Fred...,...,False,False,False,False,Day,Day,Day,Day,2,7
3,A-3629107,Source1,2017-06-13 07:14:33,2017-06-13 13:14:33,39.391690,-77.322670,39.393870,-77.326500,0.254,At MD-144/Exit 59 - Accident.,...,False,False,False,False,Day,Day,Day,Day,2,6
4,A-3051370,Source2,2018-01-08 07:48:00,2018-01-08 08:16:31,39.387390,-77.143936,NaN,NaN,0.000,Accident on MD-808 Main St Northbound at Ellis...,...,False,False,False,False,Day,Day,Day,Day,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112328,A-7557011,Source1,2018-07-05 15:05:28,2018-07-05 21:05:28,39.401610,-76.563460,39.401740,-76.567870,0.236,At MD-542/Lock Raven Blvd/Exit 29 - Accident. ...,...,False,False,False,False,Day,Day,Day,Day,3,7
112329,A-3629913,Source1,2017-06-15 14:33:24,2017-06-15 20:33:24,38.819500,-76.917669,38.820156,-76.922780,0.279,At MD-5/Branch Ave/Exit 7 - Accident.,...,False,False,False,False,Day,Day,Day,Day,3,6
112330,A-381693,Source2,2017-03-14 04:31:38,2017-03-14 05:18:32,38.960472,-76.919800,NaN,NaN,0.010,Accident on MD-201 Kenilworth Ave Southbound a...,...,False,False,True,False,Night,Night,Night,Night,2,3
112331,A-7463123,Source1,2019-01-15 08:49:03,2019-01-15 09:18:29,39.026619,-76.615523,39.021964,-76.615013,0.323,At US-50/US-301 - Accident. Lane blocked.,...,False,False,False,False,Day,Day,Day,Day,2,1


In [ ]:
# Use month column to create season column:
# Dec/Jan/Feb: Winter
# Mar/Apr/May: Spring
# Jun/Jul/Aug: Summer
# Sep/Oct/Nov: Fall
def season(x):
    tomap = {1:"Winter",2:"Winter",3:"Spring",4:"Spring",5:"Spring",6:"Summer",7:"Summer",8:"Summer",9:"Fall",10:"Fall",11:"Fall",12:"Winter"}
    if x in tomap:
        return tomap[x]
    return x

In [ ]:
# Pandas giving data issues, confirmation that month is created and datetime
df['Start_Time'] = pd.to_datetime(df['Start_Time'],format='mixed')
df['Month'] = df['Start_Time'].dt.month

# Apply season function based on month col
df["Season"] = df.Month.apply(season)
df['Season'].value_counts()

Season
Winter    32252
Fall      30521
Spring    25026
Summer    24534
Name: count, dtype: int64

In [14]:
# Convert street names to strings
df['Street']=df['Street'].astype(str)

In [ ]:
# Added misc to handle any names that did not fit into the below categories, typos, misspellings, unsure what street
def street_type(x):
    if "Tunl" in x or "Tunnel" in x:
        return "Tunnel"#nsew(x,"Tunnel")
    if "I-" in x:
        return "Interstate"
    if " Dr " in x or " Ave " in x or " Rd " in x or " Xing" in x or " Way " in x:
        return "local"
    if "Beltway" in x:
        return "Beltway"
    if "US-" in x[:4] or "MD-" in x[:4] or "Expy" in x:
        return "Highway"
    if "Hwy" in x or "Fwy" in x or "Highway" in x or "Route" in x or "Connector" in x or "Byp" in x or "Bypass" in x or "Gtwy" in x or "Brg" in x or "Trwy" in x:
        return "Highway"
    if x[-2:] in ["St","Pl","Dr","Sq"]:
        return "local"
    if "Ln" in x or "Rd" in x or "Ct" in x:
        return "local"
    if x[-3:] in ["Way","Cir","Ter","Trl","Plz","Aly","Grn","way","Ext"]:
        return "local"
    if "Ave" in x[-5:] or " Blvd" in x or " Pike" in x or "Pkwy" in x or "Crse" in x or "Alameda" in x:
        return "large_local"
    if "Garth" in x or "Spell" in x or "Psge" in x or "Chase" in x or "Cutoff" in x or "Spire" in x:
        return "local"
    if "Overlook" in x or "Battlefield" in x or "Cemetery" in x or "Park" in x or "Base" in x or "Concourse" in x or "Memorial" in x or "Booth's" in x:
        return "special"
    if "Loop" in x or "Greenway" in x or "West" in x or "ville" in x or "wood" in x:
        return "local"
    else:
        return "misc"
    return x

In [ ]:
# Apply street categories to street name col
df["Street"] = df.Street.apply(street_type)

In [ ]:
# Remove rows with "special" or "misc" street names
df=df.loc[~df.Street.str.contains("special") & ~df.Street.str.contains("misc")]


In [18]:
df.Street.value_counts()

Street
Interstate     31033
local          27577
large_local    19732
Highway        19697
Beltway        11484
Tunnel          1747
Name: count, dtype: int64

In [ ]:
# Proper formatting of county names (I took an educated guess that \"Baltimore\" referred to Balt. County and not City, this
# may not be correct)
# Also apostrophes needed to be removed to work with Weka for binning after cleaning data

def replace_misspelled(text):
    corrections = {"St Mary's": "St. Marys", "Prince George's": "Prince Georges", "Queen Anne's": "Queen Annes",
                   "Baltimore (City)":"Baltimore City","Saint Mary's":"St Marys", "Baltimore County":"Baltimore",
                   "St. Mary's":"St Marys", "Saint Marys":"St Marys"}
    for word, correction in corrections.items():
        text = text.replace(word, correction)
    return text

# Apply spelling function to data
df.loc[:,'County'] = df['County'].apply(replace_misspelled)
df["County"].value_counts(sort=True)

County
Prince Georges          21537
Montgomery              17872
Baltimore               17594
Frederick               10395
Anne Arundel             9372
Baltimore City           8052
Harford                  5421
Howard                   4939
Washington               3881
Cecil                    1983
Queen Annes              1929
Charles                  1624
Carroll                  1099
Talbot                    944
Worcester                 898
Caroline                  725
Allegany                  618
St Marys                  524
Wicomico                  494
Dorchester                477
Garrett                   291
Kent                      258
Somerset                  171
Calvert                   169
District Of Columbia        3
Name: count, dtype: int64

In [20]:
# One hot function (Could not find the one line function Dr. Olsen referred to)
def onehotinator(df, string):
    one_hot = pd.get_dummies(df[string], prefix=string)
    one_hot = one_hot.astype(int)
    df = df.join(one_hot)
    df = df.drop(columns=string)
    return df

In [ ]:
# Boolean column feature for if snowy OR icy conditions were present
# Tried to fix the warning about setting value to a DF slice, was not able to fix
lst = ['Snow', 'Freezing', 'Ice', 'Wintry', 'Hail', 'Sleet']
df['Snow_Ice']=np.where(df.loc[:, 'Weather_Condition'].str.contains('|'.join(lst), na=False), True, False)

C:\Users\bkb3\AppData\Local\Temp\ipykernel_6408\3813512010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Snow_Ice']=np.where(df.loc[:, 'Weather_Condition'].str.contains('|'.join(lst), na=False), True, False)


In [22]:
# Boolean column feature for if mist OR fog OR haze was present, OR if visibility in miles was less than 0.001 mi
lst = ['Fog', 'Mist', 'Haze', 'Heavy']
df.loc[:, 'Low_Vis_Weather']=np.where((df.loc[:, 'Weather_Condition'].str.contains('|'.join(lst), na=False) | df.loc[:, "Visibility(mi)"] < 0.001), True, False)

C:\Users\bkb3\AppData\Local\Temp\ipykernel_6408\2459017259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Low_Vis_Weather']=np.where((df.loc[:, 'Weather_Condition'].str.contains('|'.join(lst), na=False) | df.loc[:, "Visibility(mi)"] < 0.001), True, False)


In [23]:
# Boolean column feature for if road conditions were wet (weather was rain/storm related, OR there was more than 0 inches
# of precipitation at time of crash)
lst = ['Rain', 'Drizzle', 'Thunderstorm', 'T-Storm', 'Precipitation']
df.loc[:, 'Rainy']=np.where((df['Weather_Condition'].str.contains('|'.join(lst), na=False) | df['Precipitation(in)'] >= 0.01), True, False)

C:\Users\bkb3\AppData\Local\Temp\ipykernel_6408\1820217445.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Rainy']=np.where((df['Weather_Condition'].str.contains('|'.join(lst), na=False) | df['Precipitation(in)'] >= 0.01), True, False)


In [24]:
# Check which columns to delete
df


,ID,Source,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Severity,Month,Season,Snow_Ice,Low_Vis_Weather,Rainy
0,A-1582799,Source2,2020-01-09 21:03:06,2020-01-09 21:30:52,38.909382,-76.725899,NaN,NaN,0.000,Accident on Pointer Ridge Dr at Philmont Ln.,...,Night,Night,Night,Night,2,1,Winter,False,False,False
1,A-480671,Source2,2017-06-08 08:01:59,2017-06-08 08:46:37,39.287880,-76.533447,NaN,NaN,0.000,Right hand shoulder closed due to accident on ...,...,Day,Day,Day,Day,3,6,Summer,False,True,False
2,A-259111,Source2,2016-07-14 17:13:19,2016-07-14 18:43:19,39.109459,-77.160103,NaN,NaN,0.000,Accident on MD-355 Frederick Rd at MD-355 Fred...,...,Day,Day,Day,Day,2,7,Summer,False,False,False
3,A-3629107,Source1,2017-06-13 07:14:33,2017-06-13 13:14:33,39.391690,-77.322670,39.393870,-77.326500,0.254,At MD-144/Exit 59 - Accident.,...,Day,Day,Day,Day,2,6,Summer,False,False,False
4,A-3051370,Source2,2018-01-08 07:48:00,2018-01-08 08:16:31,39.387390,-77.143936,NaN,NaN,0.000,Accident on MD-808 Main St Northbound at Ellis...,...,Day,Day,Day,Day,2,1,Winter,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112328,A-7557011,Source1,2018-07-05 15:05:28,2018-07-05 21:05:28,39.401610,-76.563460,39.401740,-76.567870,0.236,At MD-542/Lock Raven Blvd/Exit 29 - Accident. ...,...,Day,Day,Day,Day,3,7,Summer,False,False,False
112329,A-3629913,Source1,2017-06-15 14:33:24,2017-06-15 20:33:24,38.819500,-76.917669,38.820156,-76.922780,0.279,At MD-5/Branch Ave/Exit 7 - Accident.,...,Day,Day,Day,Day,3,6,Summer,False,True,False
112330,A-381693,Source2,2017-03-14 04:31:38,2017-03-14 05:18:32,38.960472,-76.919800,NaN,NaN,0.010,Accident on MD-201 Kenilworth Ave Southbound a...,...,Night,Night,Night,Night,2,3,Spring,False,True,False
112331,A-7463123,Source1,2019-01-15 08:49:03,2019-01-15 09:18:29,39.026619,-76.615523,39.021964,-76.615013,0.323,At US-50/US-301 - Accident. Lane blocked.,...,Day,Day,Day,Day,2,1,Winter,False,False,False


In [ ]:
# Remove all but one day/night column

df = df.drop(columns=['Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Weather_Condition'])


df['Sunrise_Sunset'] = df['Sunrise_Sunset'].astype("string")

#df = df.dropna(subset = ['Sunrise_Sunset'], axis = 0)
df['Sunrise_Sunset'].value_counts()

Sunrise_Sunset
Day      74428
Night    36610
Name: count, dtype: Int64

In [26]:
# Reference for data conversion
df.dtypes

ID                           object
Source                       object
Start_Time           datetime64[ns]
End_Time             datetime64[ns]
Start_Lat                   float64
Start_Lng                   float64
End_Lat                     float64
End_Lng                     float64
Distance(mi)                float64
Description                  object
Street                       object
City                         object
County                       object
State                        object
Zipcode                      object
Country                      object
Timezone                     object
Airport_Code                 object
Weather_Timestamp            object
Temperature(F)              float64
Wind_Chill(F)               float64
Humidity(%)                 float64
Pressure(in)                float64
Visibility(mi)              float64
Wind_Direction               object
Wind_Speed(mph)             float64
Precipitation(in)           float64
Amenity                     

In [ ]:
# Convert all boolean columns to 1/0 integer columns and day/night column to 1/0

bools = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 
         'Traffic_Signal', 'Turning_Loop', 'Low_Vis_Weather', 'Snow_Ice', 'Rainy']

for i in bools:
    df = df.astype({i:int})
df['Sunrise_Sunset'] = df["Sunrise_Sunset"].map({'Day':1,'Night':0})
df.dtypes

ID                           object
Source                       object
Start_Time           datetime64[ns]
End_Time             datetime64[ns]
Start_Lat                   float64
Start_Lng                   float64
End_Lat                     float64
End_Lng                     float64
Distance(mi)                float64
Description                  object
Street                       object
City                         object
County                       object
State                        object
Zipcode                      object
Country                      object
Timezone                     object
Airport_Code                 object
Weather_Timestamp            object
Temperature(F)              float64
Wind_Chill(F)               float64
Humidity(%)                 float64
Pressure(in)                float64
Visibility(mi)              float64
Wind_Direction               object
Wind_Speed(mph)             float64
Precipitation(in)           float64
Amenity                     

In [ ]:
# Function to tally number of traffic speed changing elements present at location of accident
def slowdown_level(row):
    return sum([row['Bump'],row["Crossing"], row["Give_Way"], row["Junction"], row["Railway"], row['Roundabout'], \
                row["Stop"], row["Traffic_Calming"], row["Traffic_Signal"], row["Turning_Loop"]])
    
df['Qty_Slowing_Elements'] = df.apply(slowdown_level, axis=1)

df['Qty_Slowing_Elements'].value_counts()

Qty_Slowing_Elements
0    81405
1    24472
2     5021
3      365
4        7
Name: count, dtype: int64

In [29]:
df.head()

,ID,Source,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Severity,Month,Season,Snow_Ice,Low_Vis_Weather,Rainy,Qty_Slowing_Elements
0,A-1582799,Source2,2020-01-09 21:03:06,2020-01-09 21:30:52,38.909382,-76.725899,NaN,NaN,0.000,Accident on Pointer Ridge Dr at Philmont Ln.,...,0,0,0.0,2,1,Winter,0,0,0,0
1,A-480671,Source2,2017-06-08 08:01:59,2017-06-08 08:46:37,39.287880,-76.533447,NaN,NaN,0.000,Right hand shoulder closed due to accident on ...,...,0,0,1.0,3,6,Summer,0,1,0,0
2,A-259111,Source2,2016-07-14 17:13:19,2016-07-14 18:43:19,39.109459,-77.160103,NaN,NaN,0.000,Accident on MD-355 Frederick Rd at MD-355 Fred...,...,0,0,1.0,2,7,Summer,0,0,0,1
3,A-3629107,Source1,2017-06-13 07:14:33,2017-06-13 13:14:33,39.391690,-77.322670,39.39387,-77.3265,0.254,At MD-144/Exit 59 - Accident.,...,0,0,1.0,2,6,Summer,0,0,0,0
4,A-3051370,Source2,2018-01-08 07:48:00,2018-01-08 08:16:31,39.387390,-77.143936,NaN,NaN,0.000,Accident on MD-808 Main St Northbound at Ellis...,...,0,0,1.0,2,1,Winter,0,0,0,0


In [30]:
# Reference for what to do next
df.isnull().sum()

ID                          0
Source                      0
Start_Time                  0
End_Time                    0
Start_Lat                   0
Start_Lng                   0
End_Lat                 33522
End_Lng                 33522
Distance(mi)                0
Description                 0
Street                      0
City                        8
County                      0
State                       0
Zipcode                   209
Country                     0
Timezone                  209
Airport_Code              262
Weather_Timestamp       10612
Temperature(F)          12365
Wind_Chill(F)           37752
Humidity(%)             12447
Pressure(in)            11410
Visibility(mi)          21637
Wind_Direction          15875
Wind_Speed(mph)         27125
Precipitation(in)       39350
Amenity                     0
Bump                        0
Crossing                    0
Give_Way                    0
Junction                    0
No_Exit                     0
Railway   

In [ ]:
# Column for potentially freezing conditions if temp at accident time was below freezing OR windchill was below freezing
# OR if the previous column checking for snow and ice was true regarding snow/ice present

def freezing(row):
   if (row["Temperature(F)"]<=32) | (row["Wind_Chill(F)"]<=32) | (row["Snow_Ice"] == 1):
      return 1
   else:
      return 0
df['Freezing'] = df.apply(freezing, axis=1)
df["Freezing"].value_counts()

Freezing
0    96255
1    15015
Name: count, dtype: int64

In [32]:
# Due to a technical ctrl+z error, I lost all instances of removing columns as I went, this is a quick fix (I didn't remember
# which columns were removed at which points)
df = df.drop(columns=['ID','Source','Start_Time','End_Time','Start_Lat','Start_Lng','End_Lat','End_Lng','Weather_Timestamp',
                      'Temperature(F)','Wind_Chill(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Direction',
                      'Wind_Speed(mph)','Precipitation(in)','Description','Street','City','State','Zipcode','Country','Timezone',
                      'Airport_Code','Month'])

# Dropping remaining null values left

df.isnull().sum()
df


,Distance(mi),County,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,...,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Severity,Season,Snow_Ice,Low_Vis_Weather,Rainy,Qty_Slowing_Elements,Freezing
0,0.000,Prince Georges,0,0,0,0,0,0,0,0,...,0,0,0.0,2,Winter,0,0,0,0,1
1,0.000,Baltimore City,0,0,0,0,0,0,0,0,...,0,0,1.0,3,Summer,0,1,0,0,0
2,0.000,Montgomery,0,0,1,0,0,0,0,0,...,0,0,1.0,2,Summer,0,0,0,1,0
3,0.254,Frederick,0,0,0,0,0,0,0,0,...,0,0,1.0,2,Summer,0,0,0,0,0
4,0.000,Carroll,0,0,0,0,0,0,0,0,...,0,0,1.0,2,Winter,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112328,0.236,Baltimore,0,0,0,0,0,0,0,0,...,0,0,1.0,3,Summer,0,0,0,0,0
112329,0.279,Prince Georges,0,0,0,0,0,0,0,0,...,0,0,1.0,3,Summer,0,1,0,0,0
112330,0.010,Prince Georges,0,0,0,0,0,0,0,0,...,1,0,0.0,2,Spring,0,1,0,1,0
112331,0.323,Anne Arundel,0,0,0,0,0,0,0,0,...,0,0,1.0,2,Winter,0,0,0,0,1


In [33]:
df = df.dropna(how='any')

In [ ]:
df.dtypes

# Somehow sun up/down col was a float, quick fix to be int
df.loc[:, "Sunrise_Sunset"]=df["Sunrise_Sunset"].astype(int)
df.dtypes

Distance(mi)            float64
County                   object
Amenity                   int64
Bump                      int64
Crossing                  int64
Give_Way                  int64
Junction                  int64
No_Exit                   int64
Railway                   int64
Roundabout                int64
Station                   int64
Stop                      int64
Traffic_Calming           int64
Traffic_Signal            int64
Turning_Loop              int64
Sunrise_Sunset          float64
Severity                  int64
Season                   object
Snow_Ice                  int64
Low_Vis_Weather           int64
Rainy                     int64
Qty_Slowing_Elements      int64
Freezing                  int64
dtype: object

In [35]:
df=df.drop_duplicates()

In [36]:
# Reference for thinking about CFD based division (We have 4 classes, and CFD only takes 2 at a time)
df['Severity'].value_counts()

Severity
2    48784
4     7319
3     5658
1      193
Name: count, dtype: int64

In [37]:
df.columns

Index(['Distance(mi)', 'County', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Severity', 'Season', 'Snow_Ice', 'Low_Vis_Weather', 'Rainy',
       'Qty_Slowing_Elements', 'Freezing'],
      dtype='object')

In [ ]:
# Function to break apart and export all one vs many variations of the dataset

def feat_eng_train_class_merge(df):
    cols = ['Distance(mi)', 'County', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Season', 'Snow_Ice', 'Low_Vis_Weather', 'Rainy', 'Qty_Slowing_Elements', 
       'Freezing']
    df['Class'] = np.where(df['Severity'] == 1, 1, 0)
    df_c1 = df[df['Class'] != 0]
    df_c0 = df[df['Class'] != 1]
    df_c1 = df_c1.drop_duplicates()
    df_c0 = df_c0.drop_duplicates()
    df_c1.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class1.csv", columns = cols, index = False)
    df_c0.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class234_merge.csv", columns = cols, index = False)

    df['Class'] = np.where(df['Severity'] == 2, 1, 0)
    df_c1 = df[df['Class'] != 0]
    df_c0 = df[df['Class'] != 1]
    df_c1 = df_c1.drop_duplicates()
    df_c0 = df_c0.drop_duplicates()
    df_c1.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class2.csv", columns = cols, index = False)
    df_c0.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class134_merge.csv", columns = cols, index = False)

    df['Class'] = np.where(df['Severity'] == 3, 1, 0)
    df_c1 = df[df['Class'] != 0]
    df_c0 = df[df['Class'] != 1]
    df_c1 = df_c1.drop_duplicates()
    df_c0 = df_c0.drop_duplicates()
    df_c1.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class3.csv", columns = cols, index = False)
    df_c0.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class124_merge.csv", columns = cols, index = False)

    df['Class'] = np.where(df['Severity'] == 4, 1, 0)
    df_c1 = df[df['Class'] != 0]
    df_c0 = df[df['Class'] != 1]
    df_c1 = df_c1.drop_duplicates()
    df_c0 = df_c0.drop_duplicates()
    df_c1.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class4.csv", columns = cols, index = False)
    df_c0.to_csv("../Accident_Data_Preprocessing/Data_Cleaned/feat_eng_datasets_unbin/Feat_Eng_Merge/feat_eng_class123_merge.csv", columns = cols, index = False)
    
    df = df.drop(columns = 'Class')
    return df

In [ ]:
# DO NOT RUN WITH TEST DATA CLEANING
#df = feat_eng_train_class_merge(df)


In [40]:
df.shape

(61954, 23)

In [ ]:
# Function to re-separate cleaned class column from the cleaned rest of the dataset, and export both separately into the same folder
def data_output(df, folder1, folder2, file_x, file_y):
    col_class = ['Class']
    col_data = ['Distance(mi)', 'County', 'Amenity', 'Bump', 'Crossing', 'Give_Way',
       'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop',
       'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset',
       'Season', 'Snow_Ice', 'Low_Vis_Weather', 'Rainy',
       'Qty_Slowing_Elements', 'Freezing']
    
    df = df.rename(columns = {'Severity' : 'Class'})
    df = df.drop_duplicates()
    
    X = df.loc[:, df.columns != 'Class']
    y = df.Class
    
    X.to_csv(f'../Accident_Data_Preprocessing/{folder1}/{folder2}/{file_x}.csv', columns = col_data, index = False)
    y.to_csv(f'../Accident_Data_Preprocessing/{folder1}/{folder2}/{file_y}.csv', columns = col_class, index = False)

In [42]:
# TRAIN OUTPUT
data_output(df, "Data_Cleaned", "train_data", "X_train_feat_eng", "y_train_feat_eng")

In [43]:
# TEST OUTPUT
#data_output(df, "Data_Cleaned", "test_data", "X_test_feat_eng", "y_test_feat_eng")